In [1]:
pip install ccxt

Note: you may need to restart the kernel to use updated packages.


In [2]:
import ccxt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime as dt
import plotly.graph_objects as go
from datetime import timedelta
import plotly.io as pio

In [3]:
# initialise exchanges
binance = ccxt.binance()
okx = ccxt.okex()
bitstamp = ccxt.bitstamp()
bitfinex = ccxt.bitfinex()
kraken = ccxt.kraken()

In [6]:
from datetime import datetime as dt, timedelta
import ccxt

# Convert a date to a timestamp in milliseconds
def date_to_milliseconds(date_str):
    epoch = dt.utcfromtimestamp(0)
    dt_obj = dt.strptime(date_str, "%Y-%m-%d %H:%M:%S")
    return int((dt_obj - epoch).total_seconds() * 1000)

def get_data(exchange, symbol, start_dt, end_dt, interval):     # interval inputted as a timedelta
    start_dt_str = start_dt.strftime("%Y-%m-%d %H:%M:%S")
    since = date_to_milliseconds(start_dt_str)
    duration = end_dt - start_dt
    limit = int(duration / interval)
    
    # Extract interval in the format ccxt expects (like '30m', '1h', etc.)
    minutes = interval.total_seconds() / 60
    if minutes == 60:
        timeframe = '1h'
    else:
        timeframe = '{}m'.format(int(minutes))
    
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since=since, limit=limit)
    return ohlcv

symbol = 'BTC/USDT'
start_dt = dt(2023, 8, 29)
end_dt = dt(2023, 8, 29,12)

binance_data = get_data(exchange=bitstamp, symbol=symbol, start_dt=start_dt, end_dt=end_dt, interval=timedelta(minutes=30))
okx_data = get_data(exchange=bitfinex, symbol=symbol, start_dt=start_dt, end_dt=end_dt, interval=timedelta(minutes=30))


In [7]:
def to_df(data):
    unix, op, hi, lo, cl, vol = [], [], [], [], [], []
    for i in data:
        unix.append(i[0])
        op.append(i[1])
        hi.append(i[2]) 
        lo.append(i[3]) 
        cl.append(i[4]) 
        vol.append(i[5])


    # converting unix to python datetime:
    def to_date_time(unix):
        dt_object = dt.fromtimestamp(unix/1000)
        return dt_object


    date_time = list(map(to_date_time,unix))


    index = ['','open','high','low','close','volume']
    dic = {'Date_Time':date_time,'Open': op,'High':hi,'Low': lo,'Close':cl,'Volume':vol}
    df = pd.DataFrame(dic)

    df['Times']=df['Date_Time'].dt.time
    df['Dates'] = df['Date_Time'].dt.date
    df['Vol$'] = df['Volume']*df['Close']
    df = df.groupby('Times',as_index = False)['Vol$'].mean()
#     df.groupby('Times').agg({'Vol$':'mean','Close':'last'}).reset_index()
    df['Cum_vol$'] = df['Vol$'].cumsum()
    
    return df

bin_vol = to_df(binance_data)
okx_vol = to_df(okx_data)


In [59]:
# start_time = '2023-08-01 00:00:00'
# start_timestamp = int(dt.strptime(start_time, '%Y-%m-%d %H:%M:%S').timestamp())

# # Fetch OHLCV data
# ohlcv = okx.fetch_ohlcv('BTC/USDT', '30m', since=start_timestamp, limit=100)
# len(ohlcv)

0

In [8]:
compare = pd.DataFrame()
compare['Times'] = bin_vol['Times']
compare['diff_vol'] = bin_vol['Cum_vol$']-okx_vol['Cum_vol$']
compare['ratio_vol'] = bin_vol['Cum_vol$']/okx_vol['Cum_vol$']
# compare

In [11]:
vol = go.Figure()
vol.add_trace(go.Scatter(x = bin_vol['Times'],y = bin_vol['Cum_vol$'],name = 'Binance'))
vol.add_trace(go.Scatter(x = okx_vol['Times'],y = okx_vol['Cum_vol$'],name = 'OKX'))
vol.update_layout(
    title_text="Notional Traded",  # Update the title
    xaxis_title='Time',  # Update the x-axis label
    yaxis_title='Notional Traded (USD)',width = 1300,height = 550)
vol.show()

comparison = go.Figure()
comparison.add_trace(go.Scatter(x = compare['Times'], y = compare['ratio_vol'],name = 'ratio'))
comparison.update_layout(
    title_text="Ratio of volumes",  # Update the title
    xaxis_title='Time',  # Update the x-axis label
    yaxis_title='Ratio of volumes',width = 1300,height = 550)
comparison.show()




In [10]:
price = go.Figure()
price.add_trace(go.Scatter(x = bin_price['Date_Time'],y = bin_price['Close'],name = 'Binance'))
price.add_trace(go.Scatter(x = bin_price['Date_Time'],y = okx_price['Close'],name = 'OKX'))
price.update_layout(
    title_text="BTC price on Binance and OKX",  # Update the title
    xaxis_title='Time',  # Update the x-axis label
    yaxis_title='SOLUSD',width = 1300,height = 550)
price.show()



comp = go.Figure()
comp.add_trace(go.Scatter(x = compare_p['Date_Time'],y = compare_p['percentage_diff']))
# comp.add_trace(go.Scatter(x = compare_p['Date_Time'],y = compare_p['price_diff']))
comp.update_layout(
    title_text="Comparison",  # Update the title
    xaxis_title='Time',  # Update the x-axis label
    yaxis_title='SOLUSD percentage diff(USD)',width = 1300,height = 550)

comp.show()


NameError: name 'bin_price' is not defined

In [ ]:
# symbol = 'SOL/USDT'
# start_dt = dt(2023, 8, 29)
# end_dt = dt(2023, 8, 29,12)

['BTC/USDT',
 'ETH/USDT',
 'BNB/USDT',
 'XRP/USDT',
 'USDC/USDT',
 'STETH/USDT',
 'ADA/USDT',
 'DOGE/USDT',
 'SOL/USDT',
 'TRX/USDT',
 'TON/USDT',
 'DOT/USDT',
 'MATIC/USDT',
 'LTC/USDT',
 'SHIB/USDT',
 'WBTC/USDT',
 'BCH/USDT',
 'DAI/USDT',
 'AVAX/USDT',
 'LEO/USDT']

In [13]:
start_dt = dt(2023,7,10)
end_dt = dt(2023,7,29)

symbol = 'BTC/USDT'

binance_data = get_data(exchange=binance, symbol=symbol, start_dt=start_dt, end_dt=end_dt, interval=timedelta(minutes=30))
okx_data = get_data(exchange=okx, symbol=symbol, start_dt=start_dt, end_dt=end_dt, interval=timedelta(minutes=30))
# binance_data

In [185]:
# bin_price = to_df_price(binance_data)
# okx_price = to_df_price(okx_data)

# for symbol in tickers:
symbol = 'BTC/USDT'
binance_data = get_data(exchange=binance, symbol=symbol, start_dt=start_dt, end_dt=end_dt, interval=timedelta(minutes=30))
okx_data = get_data(exchange=okx, symbol=symbol, start_dt=start_dt, end_dt=end_dt, interval=timedelta(minutes=30))

binance_price = to_df_price(binance_data)
okx_price = to_df_price(okx_data)
    
binance_price

NetworkError: binance GET https://api.binance.com/api/v3/klines?interval=30m&limit=912&symbol=BTCUSDT&startTime=1688947200000

In [184]:
len(okx_price)

100

In [168]:
table = pd.DataFrame({'Ticker':tickers})
table

,Ticker
0,BTC/USDT
1,ETH/USDT
2,BNB/USDT
3,XRP/USDT
4,USDC/USDT
5,STETH/USDT
6,ADA/USDT
7,DOGE/USDT
8,SOL/USDT
9,TRX/USDT
